This program cleans the data on the prediction of well functionality.

In [1]:
import pandas as pd
import numpy as np
import os
import pdb
from tqdm import tqdm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import collections
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#os.chdir("C:\\Users\\Xiaotong\\Desktop\\wells")

In [3]:
def load():
    df1 = pd.read_csv("training.csv", parse_dates=["date_recorded"])
    df2 = pd.read_csv("labels.csv")
    df3 = pd.read_csv("test.csv", parse_dates=["date_recorded"])
    df =pd.merge(df2, df1, on="id", how="left")
    return df3, df

In [4]:
col = "installer"

In [5]:
train, test = load()

In [6]:
def fill_nan(df):
    df[col] = df[col].fillna("0")
    df[col] = df[col].str.lower()
    df[col] = df[col].replace(["not know", "not kno"], "0")
    return df

In [7]:
train = fill_nan(train)
test = fill_nan(test)

In [8]:
train["installer"].head()

0          dmdd
1           dwe
2             0
3    finn water
4        bruder
Name: installer, dtype: object

In [9]:
def get_composite_names(df, col):
    # list of full name entities
    fullnames_dic = {x: '' for x in df[col] if len(x.split())>=2}
    fullnames_list = [x for x in df[col] if len(x.split())>=2]
    for key in fullnames_dic:
        for i in range(len(key.split())):
            fullnames_dic[key] += key.split()[i][0]
    counter = collections.Counter(fullnames_list)
    return fullnames_dic, fullnames_list, counter

In [10]:
train_names_dic, train_names_list, train_names_counter = get_composite_names(train, col)
test_names_dic, test_names_list, test_names_counter = get_composite_names(test, col)

In [11]:
def fuzzymatch_one(counter, tol_partial, tol_sort, tol_set):
    """ match composite names duplicate, starting with the most frequent ones"""
    sorted_keys_desc = [x[0] for x in counter.most_common()]
    sorted_keys_asc = [x[0] for x in sorted(counter.items(), key=lambda x: x[1])]
    matched = {}
    fuzzdic = {}
    for key in tqdm(sorted_keys_desc):
        if (key not in matched) and (key not in fuzzdic):
            matched[key] = [(key, counter[key])]
            fuzzdic[key] = (key, key, counter[key])
            for item in sorted_keys_asc:
                if (item != key) and (item not in fuzzdic):
                    p0 = fuzz.ratio(key, item)
                    p1 = fuzz.partial_ratio(key, item)
                    p2 = fuzz.token_sort_ratio(key, item)
                    p3 = fuzz.token_set_ratio(key, item)
                    if (p1>tol_partial and p3>tol_set) or (p2>tol_sort and p3>tol_set):
                        matched[key].append((item, counter[item]))
                        fuzzdic[item] = (key, item, counter[item])
    return matched, fuzzdic

In [12]:
def hand_match_method1(matched, fuzzdic, key_list, value):
    for key in key_list:
        fuzzdic[key] = (value, key, fuzzdic[key][2]) 
        matched[value].append((key, fuzzdic[key][2]))
        if key in matched:
            for key2 in matched[key]:
                fuzzdic[key2[0]] = (value, key2[0], fuzzdic[key2[0]][2])
            del matched[key]
    return matched, fuzzdic

def hand_match_method2(matched, fuzzdic, value, string):
    for key in fuzzdic:
        if (string in fuzzdic[key][0]) and (key != value):
            fuzzdic[key] = (value, key, fuzzdic[key][2])
            matched[value].append((key, fuzzdic[key][2]))
            if key in matched:
                del matched[key]
    return matched, fuzzdic



In [13]:
train_match, train_fuzz = fuzzymatch_one(train_names_counter, 50, 50, 90)

100%|██████████| 403/403 [00:03<00:00, 106.41it/s]


In [14]:
key_list = ['rc ch', 'rc c', 'roman church', 'roman catholic', 'roman cathoric -kilomeni',
            'roman cathoric -same', 'rc cathoric', 'rulenge diocese', 'roman catholic rulenge diocese',
           'roman ca']
value = 'rc church'
train_match, train_fuzz = hand_match_method1(train_match, train_fuzz, key_list, value)

In [15]:
key_list = ['central govt', 'tanzania government', 'tcrs /government', 'concern /government', 'adra /government',
            'ministry of water', 'ministry of healthy']
value = 'central government'
train_match, train_fuzz = hand_match_method1(train_match, train_fuzz, key_list, value)

In [16]:
key_list = ['consultant engineer', 'citizen engine', 'howard and humfrey consultant']
value = 'consulting engineer'
train_match, train_fuzz = hand_match_method1(train_match, train_fuzz, key_list, value)

In [17]:
key_list = ['local te', 'local contract', 'local fundi', 'local technical tec', 'local technical']
value = 'local  technician'
train_match, train_fuzz = hand_match_method1(train_match, train_fuzz, key_list, value)

In [18]:
string = 'kkt'
value = 'kkkt church'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

In [19]:
string = 'local t'
value = 'kkkt church'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

In [20]:
string = 'italy'
value = 'italian government'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

In [21]:
string = 'would bank'
value = 'world bank'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

string = 'would vission'
value = 'world vision'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

string = 'village'
value = 'village council'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

string = 'china henan contractor'
value = 'china henan construction'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

In [22]:
check = {key: value for key, value in train_match.items() if type(value)!=str}
[x for x in check.values()]

[[('district council', 220),
  ('district  council', 1),
  ('mbunge / district council', 1),
  ('district counci', 1),
  ('mbozi district council', 2)],
 [('central government', 170),
  ('cebtral government', 1),
  ('central government/tlc', 1),
  ('central govt', 37),
  ('tanzania government', 4),
  ('tcrs /government', 1),
  ('concern /government', 2),
  ('adra /government', 1),
  ('ministry of water', 6),
  ('ministry of healthy', 1)],
 [('world vision', 168),
  ('world vision & community', 1),
  ('world vission', 2),
  ('would vission', 1)],
 [('fini water', 81), ('fin water', 23)],
 [('rc church', 55),
  ('cefa/ rc church', 1),
  ('rc churc', 3),
  ('rc church/central gover', 4),
  ('rc ch', 19),
  ('rc c', 4),
  ('roman church', 4),
  ('roman catholic', 5),
  ('roman cathoric -kilomeni', 2),
  ('roman cathoric -same', 2),
  ('rc cathoric', 1),
  ('rulenge diocese', 1),
  ('roman catholic rulenge diocese', 1),
  ('roman ca', 1)],
 [('handeni trunk main(', 52), ('handeni trunk main

In [23]:
train_fuzz

{'abdi mtili': ('abdi mtili', 'abdi mtili', 1),
 'abraham palanjo': ('abraham palanjo', 'abraham palanjo', 1),
 'action aid': ('action aid', 'action aid', 3),
 'action contre la faim': ('action contre la faim',
  'action contre la faim',
  3),
 'active kmk': ('active kmk', 'active kmk', 1),
 'active mkm': ('active mkm', 'active mkm', 7),
 'active tank co': ('active tank co', 'active tank co', 2),
 'adp busangi': ('adp busangi', 'adp busangi', 4),
 'adra /community': ('adra /community', 'adra /community', 7),
 'adra /government': ('central government', 'adra /government', 1),
 'adra/ community': ('adra /community', 'adra/ community', 4),
 'africa muslum agence tanzania': ('africa muslum agence tanzania',
  'africa muslum agence tanzania',
  1),
 'african development foundation': ('african development foundation',
  'african development foundation',
  3),
 'african mus': ('african mus', 'african mus', 1),
 'amboni plantation': ('amboni plantation', 'amboni plantation', 1),
 'american peo

In [ ]:
def match_acronyms(matched, fuzzdic, df):
    
    return

In [24]:
# train_histo = np.histogram(list(trainnames_counter.values()), bins=len(trainnames_counter.values()))
# test_histo = np.histogram(list(testnames_counter.values()), bins=len(testnames_counter.values()))

In [25]:
# plt.bar(train_histo[1][:-1], train_histo[0])
# plt.ylim((0,30))
# plt.xlim(0,100)
# plt.xlabel("number of occurences of a name")
# plt.ylabel("number of names that occur x times")

In [26]:
# plt.bar(test_histo[1][:-1], test_histo[0])
# plt.ylim((0,30))
# plt.xlim(0,100)
# plt.xlabel("number of occurences of a name")
# plt.ylabel("number of names that occur x times")